## **C Vector addition**

In [1]:
%%file vector_add_cpu.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <time.h>

#define N 2020
#define MAX_ERR 1e-6

int main(){
    double *a, *b, *out;
    double *d_a, *d_b, *d_out; 

    // Allocate host memory
    a   = (double*)malloc(sizeof(double) * N);
    b   = (double*)malloc(sizeof(double) * N);
    out = (double*)malloc(sizeof(double) * N);

    // Initialize host arrays
    for(int i = 0; i < N; i++){
        a[i] = i / 100.0;
        b[i] = (N - i) / 100.0;
    }

    for(int i = 0; i < N; i++){
        out[i] = a[i] + b[i];
    }   

    // Verification
    for(int i = 0; i < N; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
        if (i % 101 == 0)
          printf("%.2f + %.2f = %.2f\n", a[i], b[i], out[i]);
    }
    printf("PASSED\n");

    // Deallocate host memory
    free(a); 
    free(b); 
    free(out);
}

Writing vector_add_cpu.c


In [3]:
!gcc -o vector_add_cpu vector_add_cpu.c && ./vector_add_cpu

0.00 + 20.20 = 20.20
1.01 + 19.19 = 20.20
2.02 + 18.18 = 20.20
3.03 + 17.17 = 20.20
4.04 + 16.16 = 20.20
5.05 + 15.15 = 20.20
6.06 + 14.14 = 20.20
7.07 + 13.13 = 20.20
8.08 + 12.12 = 20.20
9.09 + 11.11 = 20.20
10.10 + 10.10 = 20.20
11.11 + 9.09 = 20.20
12.12 + 8.08 = 20.20
13.13 + 7.07 = 20.20
14.14 + 6.06 = 20.20
15.15 + 5.05 = 20.20
16.16 + 4.04 = 20.20
17.17 + 3.03 = 20.20
18.18 + 2.02 = 20.20
19.19 + 1.01 = 20.20
PASSED


## CUDA Vector addition

In [4]:
 %%file vector_add_cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>

#define N 2020
#define MAX_ERR 1e-6

__global__ void vector_add(double *out, double *a, double *b, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i < n)
        out[i] = a[i] + b[i];
}

int main(){
    double *a, *b, *out;
    double *d_a, *d_b, *d_out; 

    // Allocate host memory
    a   = (double*)malloc(sizeof(double) * N);
    b   = (double*)malloc(sizeof(double) * N);
    out = (double*)malloc(sizeof(double) * N);

    // Initialize host arrays
    for(int i = 0; i < N; i++){
        a[i] = i / 100.0;
        b[i] = (N - i) / 100.0;
    }

    // Allocate device memory
    cudaMalloc((void**)&d_a, sizeof(double) * N);
    cudaMalloc((void**)&d_b, sizeof(double) * N);
    cudaMalloc((void**)&d_out, sizeof(double) * N);

    // Transfer data from host to device memory
    cudaMemcpy(d_a, a, sizeof(double) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(double) * N, cudaMemcpyHostToDevice);

    // Executing kernel
    int threadsPerBlock = 1024;
    //int blocksPerGrid =(N + threadsPerBlock - 1) / threadsPerBlock;
    int blocksPerGrid = N/threadsPerBlock + (N % threadsPerBlock == 0 ? 0:1);
    vector_add<<<blocksPerGrid, threadsPerBlock>>>(d_out, d_a, d_b, N);
    
    // Transfer data back to host memory
    cudaMemcpy(out, d_out, sizeof(double) * N, cudaMemcpyDeviceToHost);
     
    // Verification
    for(int i = 0; i < N; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
        if (i % 101 == 0)
          printf("%.2f + %.2f = %.2f\n", a[i], b[i], out[i]);
    }
    printf("PASSED\n");

    // Deallocate device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);

    // Deallocate host memory
    free(a); 
    free(b); 
    free(out);
}

Writing vector_add_cuda.cu


In [5]:
!PATH=/usr/local/cuda-10.1/bin:${PATH} nvcc -o vector_add_cuda vector_add_cuda.cu && ./vector_add_cuda

0.00 + 20.20 = 20.20
1.01 + 19.19 = 20.20
2.02 + 18.18 = 20.20
3.03 + 17.17 = 20.20
4.04 + 16.16 = 20.20
5.05 + 15.15 = 20.20
6.06 + 14.14 = 20.20
7.07 + 13.13 = 20.20
8.08 + 12.12 = 20.20
9.09 + 11.11 = 20.20
10.10 + 10.10 = 20.20
11.11 + 9.09 = 20.20
12.12 + 8.08 = 20.20
13.13 + 7.07 = 20.20
14.14 + 6.06 = 20.20
15.15 + 5.05 = 20.20
16.16 + 4.04 = 20.20
17.17 + 3.03 = 20.20
18.18 + 2.02 = 20.20
19.19 + 1.01 = 20.20
PASSED


## OpenCL Vector addition

In [6]:
%%file vector_add.cl
__kernel void vector_add(__global double *a, __global double *b, __global double *out, int n) {
    
    int i = get_global_id(0);

    if(i < n)
      out[i] = a[i] + b[i];
}

Writing vector_add.cl


In [7]:
%%file vector_add_opencl.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>

#ifdef __APPLE__
#include <OpenCL/opencl.h>
#else
#include <CL/cl.h>
#endif

#define MAX_SOURCE_SIZE (0x100000)
#define N 2020
#define MAX_ERR 1e-6

int main(void) {
    // Create the two input vectors
    double *a, *b, *out;
    int n = N;
    
    // Allocate host memory
    a   = (double*)malloc(sizeof(double) * N);
    b   = (double*)malloc(sizeof(double) * N);
    out = (double*)malloc(sizeof(double) * N);

    // Initialize host arrays
    for(int i = 0; i < N; i++){
        a[i] = i / 100.0;
        b[i] = (N - i) / 100.0;
    }


    // Load the kernel source code into the array source_str
    FILE *fp;
    char *source_str;
    size_t source_size;

    fp = fopen("vector_add.cl", "r");
    if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread( source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose( fp );

    // Get platform and device information
    cl_platform_id platform_id = NULL;
    cl_device_id device_id = NULL;   
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs( platform_id, CL_DEVICE_TYPE_ALL, 1, 
            &device_id, &ret_num_devices);

    // Create an OpenCL context
    cl_context context = clCreateContext( NULL, 1, &device_id, NULL, NULL, &ret);

    // Create a command queue
    cl_command_queue command_queue = clCreateCommandQueue(context, device_id, 0, &ret);

    // Create memory buffers on the device for each vector 
    cl_mem a_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY, 
            N * sizeof(double), NULL, &ret);
    cl_mem b_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY,
            N * sizeof(double), NULL, &ret);
    cl_mem out_mem_obj = clCreateBuffer(context, CL_MEM_WRITE_ONLY, 
            N * sizeof(double), NULL, &ret);

    // Copy the lists a and b to their respective memory buffers
    ret = clEnqueueWriteBuffer(command_queue, a_mem_obj, CL_TRUE, 0,
            N * sizeof(double), a, 0, NULL, NULL);
    ret = clEnqueueWriteBuffer(command_queue, b_mem_obj, CL_TRUE, 0, 
            N * sizeof(double), b, 0, NULL, NULL);

    // Create a program from the kernel source
    cl_program program = clCreateProgramWithSource(context, 1, 
            (const char **)&source_str, (const size_t *)&source_size, &ret);

    // Build the program
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);

    // Create the OpenCL kernel
    cl_kernel kernel = clCreateKernel(program, "vector_add", &ret);

    // Set the arguments of the kernel
    ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&a_mem_obj);
    ret = clSetKernelArg(kernel, 1, sizeof(cl_mem), (void *)&b_mem_obj);
    ret = clSetKernelArg(kernel, 2, sizeof(cl_mem), (void *)&out_mem_obj);
    ret = clSetKernelArg(kernel, 3, sizeof(cl_int), (void *)&n);
    
    // Execute the OpenCL kernel on the list

    size_t local_item_size = 64; // Process in groups of 64
    int n_blocks = n/local_item_size + (n % local_item_size == 0 ? 0:1);
    size_t global_item_size = n_blocks * local_item_size;
    
    ret = clEnqueueNDRangeKernel(command_queue, kernel, 1, NULL, 
            &global_item_size, &local_item_size, 0, NULL, NULL);

    // Read the memory buffer C on the device to the local variable C
    //int *C = (int*)malloc(sizeof(int) * N);
    ret = clEnqueueReadBuffer(command_queue, out_mem_obj, CL_TRUE, 0, 
            N * sizeof(double), out, 0, NULL, NULL);

    // Display the result to the screen
    for(int i = 0; i < N; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
        if (i % 101 == 0)
          printf("%.2f + %.2f = %.2f\n", a[i], b[i], out[i]);
    }
    printf("PASSED\n");

    // Clean up
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(a_mem_obj);
    ret = clReleaseMemObject(b_mem_obj);
    ret = clReleaseMemObject(out_mem_obj);
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);
    free(a);
    free(b);
    free(out);
    return 0;
}

Writing vector_add_opencl.c


In [8]:
 !nvcc -o vector_add_opencl vector_add_opencl.c -lOpenCL && ./vector_add_opencl

0.00 + 20.20 = 20.20
1.01 + 19.19 = 20.20
2.02 + 18.18 = 20.20
3.03 + 17.17 = 20.20
4.04 + 16.16 = 20.20
5.05 + 15.15 = 20.20
6.06 + 14.14 = 20.20
7.07 + 13.13 = 20.20
8.08 + 12.12 = 20.20
9.09 + 11.11 = 20.20
10.10 + 10.10 = 20.20
11.11 + 9.09 = 20.20
12.12 + 8.08 = 20.20
13.13 + 7.07 = 20.20
14.14 + 6.06 = 20.20
15.15 + 5.05 = 20.20
16.16 + 4.04 = 20.20
17.17 + 3.03 = 20.20
18.18 + 2.02 = 20.20
19.19 + 1.01 = 20.20
PASSED
